# STAT 404 project

author:
  - Yanping (Dedoria) Wang (89845473)
  - Kevin Zhu (81805673)
  - Raojun Bo (55832919)


In [5]:
library(dplyr)
library(ggplot2)

In [2]:
#read data
food <- read.csv("data/foodDiet.csv")
head(food)

,Person,gender,Age,Height,pre.weight,Diet,weight6weeks
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,25,0,41,171,60,2,60.0
2,26,0,32,174,103,2,103.0
3,1,0,22,159,58,1,54.2
4,2,0,46,192,60,1,54.0
5,3,0,55,170,64,1,63.3
6,4,0,33,171,64,1,61.1


In [3]:
summary(food)

     Person          gender            Age            Height     
 Min.   : 1.00   Min.   :0.0000   Min.   :16.00   Min.   :141.0  
 1st Qu.:20.25   1st Qu.:0.0000   1st Qu.:32.25   1st Qu.:164.2  
 Median :39.50   Median :0.0000   Median :39.00   Median :169.5  
 Mean   :39.50   Mean   :0.4231   Mean   :39.15   Mean   :170.8  
 3rd Qu.:58.75   3rd Qu.:1.0000   3rd Qu.:46.75   3rd Qu.:174.8  
 Max.   :78.00   Max.   :1.0000   Max.   :60.00   Max.   :201.0  
   pre.weight          Diet        weight6weeks   
 Min.   : 58.00   Min.   :1.000   Min.   : 53.00  
 1st Qu.: 66.00   1st Qu.:1.000   1st Qu.: 61.85  
 Median : 72.00   Median :2.000   Median : 68.95  
 Mean   : 72.53   Mean   :2.038   Mean   : 68.68  
 3rd Qu.: 78.00   3rd Qu.:3.000   3rd Qu.: 73.83  
 Max.   :103.00   Max.   :3.000   Max.   :103.00  

| Variable Name         | Levels of the Variable      | Changes to Original Variable    |
|-----------------------|-----------------------------|-------------------------------------|
| person        | 1-78 stands for id for each person    | drop this, not useful            |
| gender        | 0 for female, 1 for male     | None            |
| Age(years old)        | 16-60    | 16-19(Adolescent), 20-39(Adults), 40-49(Middle-Aged Adults), 50-60(Elder Adults)         |
| Height(cm)     | Continuous 141cm-201cm  | Calculate BMI with pre.weight                    |
| pre.Weight (kg)  | Continuous 58kg-103kg (weight before the experiment)      | categorize them as Underweight(<18.5), Healthy(18.5 - 25), Overweight(25 - 30), Obesity(>=30)     |
| Diet         | 1(Placebo), 2(Keto), 3(Vegan)    | None            |
| weight6weeks (kg)    | Continuous 53kg-103kg (weight after 6 weeks take diet)      |     Calculate BMI with Height        |
| weightloss (kg)    | Continuous  (weight6weeks - pre.Weight)   |    None       |

In [4]:
# Create a copy of the original data
food_original <- food

# Modify the data: drop 'Person' and perform necessary transformations
food_modified <- food_original %>%
  select(-Person) %>%  # Drop 'Person' column
  mutate(
    # Recode 'gender'
    gender = factor(gender, levels = c(0, 1), labels = c("Female", "Male")),
    
    # Reclassify 'Age' into age groups
    AgeGroup = cut(
      Age,
      breaks = c(15, 19, 39, 49, 60),
      right = TRUE,
      labels = c("Adolescent", "Adults", "Middle-Aged Adults", "Elder Adults")
    ),
    
    # Convert 'Height' to meters
    Height_m = Height / 100,
    
    # Calculate BMI before the experiment
    BMI_pre = pre.weight / (Height_m^2),
    
    # Categorize BMI_pre
    BMI_category_pre = cut(
      BMI_pre,
      breaks = c(-Inf, 18.5, 25, 30, Inf),
      labels = c("Underweight", "Healthy", "Overweight", "Obesity")
    ),
    
    # Calculate BMI after 6 weeks
    BMI_post = weight6weeks / (Height_m^2),
    
    # Calculate weight loss
    weightloss = weight6weeks - pre.weight
  )

# Select required columns for the new dataset
food_new <- food_modified %>%
  select(
    gender,
    AgeGroup,
    Diet,
    BMI_pre,
    BMI_category_pre,
    BMI_post,
    weightloss
  )

# Identify numeric columns
numeric_cols <- food_new %>%
  select(where(is.numeric)) %>%
  names()

# Standardize numeric columns and add them to the new dataset
food_new <- food_new %>%
  mutate(across(all_of(numeric_cols), ~ scale(.)[,1], .names = "{col}_std"))

# View the first few rows of the new dataset
head(food_new)

# Summary statistics of the new dataset
summary(food_new)



,gender,AgeGroup,Diet,BMI_pre,BMI_category_pre,BMI_post,weightloss,Diet_std,BMI_pre_std,BMI_post_std,weightloss_std
,<fct>,<fct>,<int>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Female,Middle-Aged Adults,2,20.51913,Healthy,20.51913,0.0,-0.04731281,-1.1367460,-0.8409967,1.50691963
2,Female,Adults,2,34.02035,Obesity,34.02035,0.0,-0.04731281,2.2089119,2.6665571,1.50691963
3,Female,Adults,1,22.94213,Healthy,21.43903,-3.8,-1.27744594,-0.5363172,-0.6020131,0.01758659
4,Female,Middle-Aged Adults,1,16.27604,Underweight,14.64844,-6.0,-1.27744594,-2.1882023,-2.3661771,-0.84465885
5,Female,Elder Adults,1,22.14533,Healthy,21.90311,-0.7,-1.27744594,-0.7337680,-0.4814449,1.23256881
6,Female,Adults,1,21.88708,Healthy,20.89532,-2.9,-1.27744594,-0.7977641,-0.7432658,0.37032336


    gender                 AgeGroup       Diet          BMI_pre     
 Female:45   Adolescent        : 1   Min.   :1.000   Min.   :16.28  
 Male  :33   Adults            :39   1st Qu.:1.000   1st Qu.:22.13  
             Middle-Aged Adults:23   Median :2.000   Median :24.75  
             Elder Adults      :15   Mean   :2.038   Mean   :25.11  
                                     3rd Qu.:3.000   3rd Qu.:28.44  
                                     Max.   :3.000   Max.   :35.21  
    BMI_category_pre    BMI_post       weightloss        Diet_std       
 Underweight: 2      Min.   :14.65   Min.   :-9.200   Min.   :-1.27745  
 Healthy    :43      1st Qu.:20.93   1st Qu.:-5.550   1st Qu.:-1.27745  
 Overweight :22      Median :23.42   Median :-3.600   Median :-0.04731  
 Obesity    :11      Mean   :23.76   Mean   :-3.845   Mean   : 0.00000  
                     3rd Qu.:26.63   3rd Qu.:-2.000   3rd Qu.: 1.18282  
                     Max.   :34.02   Max.   : 2.100   Max.   : 1.18282  
  BMI_

In [6]:
# Calculate summary statistics by Diet
diet_summary <- food_new %>%
  group_by(Diet) %>%
  summarise(
    average_weight_change = mean(weightloss, na.rm = TRUE),
    median_weight_change = median(weightloss, na.rm = TRUE),
    sd_weight_change = sd(weightloss, na.rm = TRUE),
    count = n()
  ) %>%
  arrange(desc(average_weight_change))  # Arrange in descending order of average change

# View the summary
print(diet_summary)


# A tibble: 3 × 5
   Diet average_weight_change median_weight_change sd_weight_change count
  <int>                 <dbl>                <dbl>            <dbl> <int>
1     2                 -3.03                -3.30             2.52    27
2     1                 -3.3                 -3.05             2.24    24
3     3                 -5.15                -5.4              2.40    27
